In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-qda6e883
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-qda6e883
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=bf3ecad51cb12b2f01b18e010cfd390746530f3b6bee474783e5f94e424eb280
  Stored in directory: /tmp/pip-ephem-wheel-cache-poww6dge/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [6]:
%%cuda --name MIS.cu

#include "/content/drive/MyDrive/graph/coloring.h"
#include "/content/drive/MyDrive/graph/graph_d.h"

#define THREADxBLOCK 128

using namespace std;

__device__ bool result;

Coloring* graphColoring(GraphStruct *str){
  Coloring* col;
	CHECK(cudaMallocManaged(&col, sizeof(Coloring)));
	uint n = str->nodeSize;
	col->uncoloredNodes = true;

	// cudaMalloc for arrays of struct Coloring
	CHECK(cudaMallocManaged( &(col->coloring), n * sizeof(uint)));
	memset(col->coloring,0,n);

	// allocate space on the GPU for the random states
	curandState_t* states;
	uint* weigths;
	cudaMalloc((void**) &states, n * sizeof(curandState_t));
	cudaMalloc((void**) &weigths, n * sizeof(uint));
	dim3 threads ( THREADxBLOCK);
	dim3 blocks ((str->nodeSize + threads.x - 1) / threads.x, 1, 1 );
	uint seed = 0;
	init <<< blocks, threads >>> (seed, states, weigths, n);

	for (uint j = 0; j < n; j++) {
      printf("%d", weigths[j]);
  }

	bool result_h = true;
	cudaMemcpyToSymbol(result, &result_h, sizeof(bool));

	// start coloring (dyn. parall.)
	colorer <<< 1, 1 >>> (col, str, weigths);

  cudaFree(states);
	cudaFree(weigths);
  return col;
}

__global__ void findIS (Coloring* col, GraphStruct *str, uint* weights, bool *currentIS, bool *X) {
	uint idx = threadIdx.x + blockDim.x * blockIdx.x;

	if (idx >= str->nodeSize)
		return;

	if (col->coloring[idx])
		return;

	uint offset = str->cumDegs[idx];
	uint deg = str->cumDegs[idx + 1] - str->cumDegs[idx];

	bool candidate = true;
	for (uint j = 0; j < deg; j++) {
		uint neighID = str->neighs[offset + j];
		if (!X[neighID] && !col->coloring[neighID] &&
				((weights[idx] < weights[neighID]) ||
				((weights[idx] == weights[neighID]) && idx < neighID))) {
			candidate = false;
		}
	}
	if (candidate) {
		currentIS[idx] = true;
    X[idx] = true;
    for (uint j = 0; j < deg; j++) {
        uint neighID = str->neighs[offset + j];
        X[neighID] = true;
    }
	}
}

__global__ void init (uint seed, curandState_t* states, uint* numbers, uint n) {
	uint idx = blockIdx.x * blockDim.x + threadIdx.x;
	if (idx > n)
			return;
	curand_init(seed, idx, 0, &states[idx]);
	numbers[idx] = curand(&states[idx])%n*n;
}

__global__ void checkX(GraphStruct *str, bool* X) {
		uint idx = threadIdx.x + blockDim.x * blockIdx.x;
		if (idx >= str->nodeSize)
				return;

    if(X[idx] == false)
      result = true;
}

__global__ void colorMIS(Coloring* col, GraphStruct *str, bool *currentIS){
  uint idx = threadIdx.x + blockDim.x * blockIdx.x;

	if (idx >= str->nodeSize)
		return;

	if (col->coloring[idx])
		return;

  if(currentIS[idx])
      col->coloring[idx] = col->numOfColors;
	else
	 		col->uncoloredNodes = true;
}

__global__ void colorer (Coloring* col, GraphStruct *str, uint* weights) {
	dim3 threads (THREADxBLOCK);
	dim3 blocks ((str->nodeSize + threads.x - 1) / threads.x, 1, 1 );

  bool* currentIS;
	uint n = str->nodeSize;
  cudaMalloc((void**) &currentIS, n * sizeof(bool));
  memset(currentIS, false, n);

  bool* X;
  cudaMalloc((void**) &X, n * sizeof(bool));
  memset(X, false, n);

	// loop on ISs covering the graph
	col->numOfColors = 0;
	while (col->uncoloredNodes) {
		col->uncoloredNodes = false;
		col->numOfColors++;
		result = true;
		memset(currentIS, false, n);
		memset(X, false, n);

    while(result){
				result = false;
        findIS <<< blocks, threads >>> (col, str, weights, currentIS, X);

		    //cudaDeviceSynchronize();
				checkX <<< blocks, threads >>> (str, X);
    }

    colorMIS <<< blocks, threads >>> (col, str, currentIS);
	}
}


'File written in /content/src/MIS.cu'

In [4]:
%%cuda --name test_MIS.cu

#include "/content/drive/MyDrive/graph/coloring.h"

using namespace std;

int main(void) {
	unsigned int n = 10000;		 // number of nodes for random graphs
	float prob = 0.5;				    // density (percentage) for random graphs
	std::default_random_engine eng{0};  // fixed seed

  srand(time(0));
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

	// new graph with n nodes
	Graph graph(n,1);

	// generate a random graph
	graph.randGraph(prob,eng);

	// get the graph struct
	GraphStruct *str = graph.getStruct();

  cudaEventRecord(start);

	Coloring* col = graphColoring(str);
	cudaDeviceSynchronize();

  cudaEventRecord(stop);
  cudaEventSynchronize(stop);

  //Stampo in millisecondi quanto tempo ci ha messo a colorare il grafo.
  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);
  printf("%f ms\n", milliseconds);

	//printColoring(col, str, 1);



	return EXIT_SUCCESS;
}

'File written in /content/src/test_MIS.cu'

In [7]:
!nvcc -dc src/test_MIS.cu /content/src/MIS.cu /content/drive/MyDrive/graph/graph.cpp /content/drive/MyDrive/graph/graph_d.cu
!nvcc test_MIS.o MIS.o graph.o graph_d.o -o test_MIS
!./test_MIS